In [1]:
import copy
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

def last_modified(opts, remote_path):
    info = get_info_remote(get_wdclient(opts), remote_path.as_posix())
    format_ = '%a, %d %b %Y %H:%M:%S GMT'
    return datetime.datetime.strptime(info['modified'], format_)

# Macro-Pipeline AHN3 Workflow - GeoTIFF Export (Only Vegetation Points)

## Set Run-Specific Input

Fill in the username/password for the SURF dCache. Choose whether you want to i) run all input files, ii) run the only input files listed in `filename`, or iii) run the input that was updated since the last workflow run.

In [6]:
remote_path_root = pathlib.Path('/pnfs/grid.sara.nl/data/projects.nl/eecolidar/01_Escience/')

# dCache path to retiled data 
remote_path_input = remote_path_root / 'ALS/Netherlands/ahn3_current/ahn3_current_TOP10NL_ud20200323_targets_veg'

# dCache path where to copy the geotiff files
remote_path_output = remote_path_input.with_name(remote_path_input.name.replace('targets_veg', 'geotiff_veg_mean_norm_height'))

run = 'from_file' # 'all', 'updated', 'from_file'
filename = 'geotiff_export_veg_failed.json'  # if run is 'from_file', set name of file with input file names
assert run in ['all', 'updated', 'from_file']

In [3]:
webdav_login = input('WebDAV username: ')
webdav_password = getpass.getpass('WebDAV password: ')
if run == 'updated':
    last_run = datetime.datetime.strptime(input('Date last run (YYYY-MM-DD): '), '%Y-%m-%d')

WebDAV username:  fnattin4
WebDAV password:  ············


## Check Connection to Remote Storage

In [4]:
wd_opts = {
    'webdav_hostname': 'https://webdav.grid.surfsara.nl:2880',
    'webdav_login': webdav_login,
    'webdav_password': webdav_password,
    'webdav_timeout': 200
}
assert get_wdclient(wd_opts).check(remote_path_root.as_posix())

In [7]:
features = [f.strip('/') for f in list_remote(get_wdclient(wd_opts), remote_path_input.as_posix())
            if not fnmatch.fnmatch(f, 'tile_*_*.log')]
print('Found: {} features'.format(len(features)))
if run == 'updated':
    # determine which tiles have been updated since last run
    features = [f for f in features if last_modified(wd_opts, remote_path_input/f) > last_run]
elif run == 'from_file':
    with open(filename, 'r') as f:
        features_read = json.load(f)
    # check whether all files are available on dCache
    assert all([f in features for f in features_read]), f'Some of the files in {filename} are not in remote dir'
    features = features_read
print('Retrieve and extract {} features: '.format(len(features)))
print(', '.join(features))

Found: 20 features
Retrieve and extract 1 features: 
mean_normalized_height


## Setup Cluster

Setup Dask cluster used for all the macro-pipeline calculations.

In [8]:
local_tmp = pathlib.Path('/data/local/tmp')

nprocs_per_node = 1  

# start the cluster
scheduler_node = 'node1'

# hosts = [f'node{i}' for i in range(1, 11)]
hosts = ['node2']
cluster = SSHCluster(hosts=[scheduler_node] + hosts, 
                     connect_options={'known_hosts': None, 
                                      'username': 'ubuntu', 
                                      'client_keys': '/home/ubuntu/.ssh/id_rsa'},
                     worker_options={'nthreads': 1, 
                                     'nprocs': nprocs_per_node,
                                     'memory_limit': 0,
                                     'local_directory': local_tmp/'dask-worker-space'}, 
                     scheduler_options={'dashboard_address': '8787'})
cluster

distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
distributed.deploy.ssh - INFO - warnings.warn(msg)
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - Clear task state
distributed.deploy.ssh - INFO - distributed.scheduler - INFO -   Scheduler at: tcp://145.100.59.123:8786
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://145.100.59.182:45445'
distributed.deploy.ssh - INFO - /usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is inco

SSHCluster('tcp://145.100.59.123:8786', workers=0, threads=0, memory=0 B)

## GeoTIFF Export

Export the rasterized features from the target grid to GeoTIFF files.

In [9]:
# output handle: AHN3 dataset, features, target grid spacing 10m, normalization grid spacing 1m, only vegetation points
output_handle = 'ahn3_feat_10m_1m_veg'

# setup input dictionary to configure the geotiff export pipeline
# NOTE: to export the geotiffs we have mounted the dCache storage with rclone
geotiff_export_input_veg = {
    'parse_point_cloud': {},
    'data_split': {'xSub': 1, 'ySub': 1},
    'create_subregion_geotiffs': {'output_handle': output_handle},
    'pushremote': remote_path_output.as_posix(),
#     'cleanlocalfs': {}  # DO NOT CLEAN - it would erase remote input.. 
}

# write input dictionary to JSON file
with open('geotiff_export_input_veg_mean_norm_height.json', 'w') as f:
    json.dump(geotiff_export_input_veg, f)

In [10]:
macro = MacroPipeline()

for feature in features:
    gw = GeotiffWriter(bands=feature, label=feature).setup_webdav_client(wd_opts)
    geotiff_export_input_veg_ = copy.deepcopy(geotiff_export_input_veg)
    # dCache is mounted with rclone - point input to path as if it was local
    geotiff_export_input_veg_['setup_local_fs'] = {
        'input_folder': f'/data/local/home/eecolidar_webdav/01_Escience/ALS/Netherlands/ahn3_current/ahn3_current_TOP10NL_ud20200323_targets_veg/{feature}',
        'output_folder': (local_tmp/feature).as_posix()
    }
    gw.config(geotiff_export_input_veg_)
    macro.add_task(gw)

macro.setup_cluster(cluster=cluster)

# run!
macro.run()

# save outcome results and write name of failed pipelines to file
macro.print_outcome(to_file='geotiff_export_veg.out')
failed = macro.get_failed_pipelines()
if failed:
    with open('geotiff_export_veg_failed.json', 'w') as f:
        json.dump([pip.label for pip in failed], f)
    raise RuntimeError('Some of the pipelines have failed')

In [ ]:
from dask.distributed import Client, Future
client = Client('tcp://145.100.59.123:8786')
futures = [Future(key) for key in client.who_has().keys()]
client.cancel(futures)

## Terminate cluster

In [11]:
macro.shutdown()

tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://145.100.59.123:8786' processes=1 threads=1>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/usr/local/lib/python3.7/site-packages/distributed/client.py", line 1157, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/usr/local/lib/python3.7/site-packages/distributed/batched.py", line 117, in send
    raise CommClosedError
distributed.comm.core.CommClosedError
